In [1]:
import glob
import torch
import json

from detectron2.evaluation.coco_evaluation import instances_to_coco_json

In [2]:
data_dir = "../data/rockfish_full/"

In [3]:
pths = glob.glob(data_dir + "**/*.pth", recursive=True)
len(pths)

130

In [4]:
anno_id = 0
for pth in pths:
    loaded = torch.load(pth)
    
    ims = []
    annos = []

    for im in loaded:
        ims.append({
                'file_name': im['file_name'],
                'id': im['file_name'],
                'height': im["instances"]._image_size[0],
                'width': im["instances"]._image_size[1]
            })

        instances = instances_to_coco_json(im['instances'], im['file_name'])
        annos += instances
    
    for i, anno in enumerate(annos):
        anno['id'] = anno_id
        anno['category_id'] = 1 # need to map to our categories
        anno_id += 1

    print(pth, "ims", len(ims))
    print(pth, "annos", len(annos))
    
    full_coco = { 'info': {},
                 'licenses': [],
                 'categories': [ { "supercategory": "THING", "id": 1 , "name": "THING" }],
                 'images': ims,
                 'annotations': annos
                }
    
    output_fname = pth[:-4] + "_coco.json"
    json.dump(full_coco, open(output_fname, "w"))

../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-6k-1x_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-6k-1x_preds.pth annos 6435
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/en-b0-ptannos-2x.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/en-b0-ptannos-2x.pth annos 21387
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-10pct_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-10pct_preds.pth annos 119
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-6k-1x_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-6k-1x_preds.pth annos 6143
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/en-b0-ptannos-2x.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/en-b0-ptannos-2x.pth annos 28997
../data/rockfish_full/RL-16_06/d2

In [48]:
annos

[{'image_id': '../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/20181015.155525.00049_rect_color.tif',
  'category_id': 0,
  'bbox': [901.195556640625,
   61.59581756591797,
   177.562744140625,
   173.45626831054688],
  'score': 0.9853487610816956,
  'id': 724806},
 {'image_id': '../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/20181015.155525.00049_rect_color.tif',
  'category_id': 0,
  'bbox': [907.9029541015625,
   102.35740661621094,
   162.240234375,
   94.78422546386719],
  'score': 0.8004311919212341,
  'id': 724807},
 {'image_id': '../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/20181015.155533.00052_rect_color.tif',
  'category_id': 0,
  'bbox': [133.95179748535156,
   685.4609985351562,
   204.46754455566406,
   203.377197265625],
  'score': 0.6585875749588013,
  'id': 724808},
 {'image_id': '../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/20181015.155541.00055_rect_color.tif',
  'category_id': 0,
  'bbox': [824.56738